In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [4]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Embedding text column

In [5]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

vectors.shape

C:\Users\91756\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\91756\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91756\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(8, 768)

In [6]:
dim = vectors.shape[1]
dim

768

In [9]:
# Build a FAISS Index for vectors

import faiss

index = faiss.IndexFlatL2(dim)

In [11]:
# Normalize the source vectors and add to the index

index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000190800A41B0> >

### Step 2: Encode search text using same encorder and normalize the output vector

In [12]:
search_query = "I want to buy a polo t-shirt"

vec = encoder.encode(search_query)
vec.shape

(768,)

In [13]:
# Reshaping the array

import numpy as np

svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

### Step 3: Search for similar vector in the FAISS index created

In [14]:
distances,I = index.search(svec,k=2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [15]:
I

array([[3, 2]], dtype=int64)

In [16]:
I.tolist()
I

array([[3, 2]], dtype=int64)

In [17]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [18]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [19]:
search_query

'I want to buy a polo t-shirt'